In [1]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

'/Users/bernat/opt/anaconda3/envs/ironhack/lib/python3.9/site-packages/chromedriver_autoinstaller/107/chromedriver'

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import os
import numpy as np
import requests
import pymysql
import sqlalchemy as alch
from getpass import getpass

import googletrans
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy



In [3]:
#Functions!!!!!!!

#title
def get_title(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    word="dt"

    tags = soup.find_all(word)
    titles=[c.getText().replace(">", "").strip() for c in tags]
    #the first 44 items are from the menu -> we don't want this, then, titles are duplicated!!!!
    titles = titles[44:]
    newtitles=[]
    i=0
    while i < len(titles):
        newtitles.append(titles[i])
        i+=2
    return newtitles

#subtitle
def get_subtitle(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    word="dd"

    tags = soup.find_all(word, attrs={"class":"soporte"})
    subtitles=[c.getText().replace(">", "").strip() for c in tags]

    newsub=[]
    i=0
    while i < len(subtitles):
        newsub.append(subtitles[i])
        i+=2

    return newsub


#author
def get_author(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    word="dd"

    tags = soup.find_all(word, attrs={"class":"autor"})
    authors=[c.getText().replace(">", "").strip() for c in tags]
    newaut=[]
    i=0
    while i < len(authors):
        newaut.append(authors[i])
        i+=2
    return newaut

#images
def get_image_list(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    pics = soup.find_all("img")
    image_list=[]
    for i in range(1,len(pics)-7):
        image_list.append(pics[i].get("src"))    
    return image_list

In [4]:
import os

url = "https://www.museodelprado.es/en/the-collection/art-works"

options = webdriver.ChromeOptions()
options.add_argument("--incognito")
#options.add_argument("--headless")

driver = webdriver.Chrome(options = options)
driver.get(url)

WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()

#start = time.time()
driver.execute_script("document.querySelector('footer').style.display = 'None'")


#elemento = "document.querySelector('mostrable miniaturas')"
#buscador-coleccion-resultados



last_height = driver.execute_script("return document.body.scrollHeight")
html = driver.execute_script("return document.body.outerHTML;")
soup = BeautifulSoup(html, "html.parser")
total_number = soup.find_all("strong", {"id":"panNumResultados"})[0]
a=total_number.text
total_number=int(re.findall("\d{4}",a)[0])

while len(get_image_list(html)) < total_number:
    #driver.execute_script("document.querySelector('footer').style.display = 'None'")
    driver.execute_script(f"window.scrollBy(0, {last_height});")

    time.sleep(5)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height

html = driver.execute_script("return document.body.outerHTML;")
print(len(html))
os.system("say -v monica ayamdon")
#OK -> this function will give me the links of the images



19484106


0

In [ ]:
"""while True:

    driver.execute_script("document.querySelector('footer').style.display = 'None';")
    #driver.execute_script("document.querySelector('buscador-coleccion-resultados').scrollTo(0, document.querySelector('buscador-coleccion-resultados').scrollHeight);")
    #time.sleep(5)
    driver.execute_script("document.querySelector('.buscador-coleccion-resultados').scrollTo(0, document.body.scrollHeight);")
    new_height = driver.execute_script("return document.querySelector('.buscador-coleccion-resultados').scrollHeight")
    #buscador-coleccion-resultados
    print(new_height)
    if new_height == last_height:
        break
    last_height = new_height
    end = time.time()
    if round(end-start) > 70:
        break"""

In [5]:
#OK -> this function will give me the links of the images
images=get_image_list(html)
len(images)


7001

In [6]:
#subtitle
subtitles=get_subtitle(html)


In [7]:
#author

artists = get_author(html)

In [8]:
#title

titles=get_title(html)

In [9]:
print(len(images))
print(len(subtitles))
print(len(artists))
print(len(titles))

7001
7001
7001
7001


In [10]:
prado_df = pd.DataFrame({"title":titles, "subtitle":subtitles, "artist":artists, "img_link": images})

In [11]:
prado_df

,title,subtitle,artist,img_link
0,"Christ Pantocrator held by four Angels, Hermit...",Fresco painting on mural transferred to canvas...,Anonymous,https://content3.cdnprado.net/imagenes/Documen...
1,"Lower Part of God the Father, Hermitage of Ver...",Fresco painting on mural transferred to canvas...,Anonymous,https://content3.cdnprado.net/imagenes/Documen...
2,"Mary Magdalene, Hermitage of Vera Cruz, Maderu...",Fresco painting on mural transferred to canvas...,Anonymous,https://content3.cdnprado.net/imagenes/Documen...
3,"The Adoration of a Wise Man, Hermitage of Vera...",Fresco painting on mural transferred to canvas...,Anonymous,https://content3.cdnprado.net/imagenes/Documen...
4,"Two Apostles and the Remains of a Third, Hermi...",Fresco painting on mural transferred to canvas...,Anonymous,https://content2.cdnprado.net/imagenes/Documen...
...,...,...,...,...
6996,Thonet armchair,"Ratán/médula, Metal, Wood. Ca. 1930","Thonet, Michael",https://content3.cdnprado.net/imagenes/Documen...
6997,Thonet chair,"Ratán/médula, Metal, Wood. Ca. 1930","Thonet, Michael",https://content3.cdnprado.net/imagenes/Documen...
6998,Plaque commemorating the 1st centenary of Goya...,Brass. 1928,"Blay y Fábrega, Miguel",https://content3.cdnprado.net/imagenes/Documen...
6999,Circular carpet,Wool. 1928,Real Fábrica de Tapices,https://content3.cdnprado.net/imagenes/Documen...


In [12]:
# I will split the years, that are contained in 

subt=[]
year=[]
for line in prado_df["subtitle"]:
    newline= line.split(". ")
    subt.append(newline[0])
    year.append(newline[-1])

prado_df["subtitle"] = subt
prado_df["year"]= year

prado_df["year"] = [line.replace (" century","")for line in prado_df["year"]]


In [13]:
prado_df.head()

,title,subtitle,artist,img_link,year
0,"Christ Pantocrator held by four Angels, Hermit...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,XII
1,"Lower Part of God the Father, Hermitage of Ver...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,XII
2,"Mary Magdalene, Hermitage of Vera Cruz, Maderu...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,XII
3,"The Adoration of a Wise Man, Hermitage of Vera...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,XII
4,"Two Apostles and the Remains of a Third, Hermi...",Fresco painting on mural transferred to canvas,Anonymous,https://content2.cdnprado.net/imagenes/Documen...,XII


### I will transform centuries to years!

In [14]:
centuries = {"VIII":"700","IX":"800","X":"900","XI":"1000","XII":"1100","XIII":"1200","XIV":"1300","XV":"1400","XVI":"1500","XVII":"1600",
"XVIII":"1700","XIX":"1800","XX":"1900","XXI":"2000"}

In [15]:
def clean_centuries(column):
    nl0=[]
    nl=[]
    for y in column:
        if y[0]==" ":
            nl0.append(y[1:])
        else:
            nl0.append(y)

    for y in nl0:
               
        if y in centuries:
            new = centuries[y]
            nl.append(new)
        else:
            nl.append(y)
    return nl



    

In [16]:
prado_df["year"]=clean_centuries(prado_df["year"])

In [19]:
prado_df["year"]

,title,subtitle,artist,img_link,year
0,"Christ Pantocrator held by four Angels, Hermit...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,1100
1,"Lower Part of God the Father, Hermitage of Ver...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,1100
2,"Mary Magdalene, Hermitage of Vera Cruz, Maderu...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,1100
3,"The Adoration of a Wise Man, Hermitage of Vera...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,1100
4,"Two Apostles and the Remains of a Third, Hermi...",Fresco painting on mural transferred to canvas,Anonymous,https://content2.cdnprado.net/imagenes/Documen...,1100
...,...,...,...,...,...
6996,Thonet armchair,"Ratán/médula, Metal, Wood","Thonet, Michael",https://content3.cdnprado.net/imagenes/Documen...,1930
6997,Thonet chair,"Ratán/médula, Metal, Wood","Thonet, Michael",https://content3.cdnprado.net/imagenes/Documen...,1930
6998,Plaque commemorating the 1st centenary of Goya...,Brass,"Blay y Fábrega, Miguel",https://content3.cdnprado.net/imagenes/Documen...,1928
6999,Circular carpet,Wool,Real Fábrica de Tapices,https://content3.cdnprado.net/imagenes/Documen...,1928


In [55]:
#passarli un regex per a que em tregui nums!
def clean_year(col):
    nl=[]
    a=[]
    out=[]
    newout=[]
    for elem in col:
        if "-" in elem:
            nl.append(elem)
        else:
            nl.append(elem)

    for e in nl:
        a.append(e.split(" - ")[-1])
    
    for elem in a:
        out.append(re.findall("\d{4}",elem))


    return out

a=clean_year(prado_df["year"])
type(a[0][0])

str

In [39]:
prado_df

,title,subtitle,artist,img_link,year
0,"Christ Pantocrator held by four Angels, Hermit...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,1100
1,"Lower Part of God the Father, Hermitage of Ver...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,1100
2,"Mary Magdalene, Hermitage of Vera Cruz, Maderu...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,1100
3,"The Adoration of a Wise Man, Hermitage of Vera...",Fresco painting on mural transferred to canvas,Anonymous,https://content3.cdnprado.net/imagenes/Documen...,1100
4,"Two Apostles and the Remains of a Third, Hermi...",Fresco painting on mural transferred to canvas,Anonymous,https://content2.cdnprado.net/imagenes/Documen...,1100
...,...,...,...,...,...
6996,Thonet armchair,"Ratán/médula, Metal, Wood","Thonet, Michael",https://content3.cdnprado.net/imagenes/Documen...,1930
6997,Thonet chair,"Ratán/médula, Metal, Wood","Thonet, Michael",https://content3.cdnprado.net/imagenes/Documen...,1930
6998,Plaque commemorating the 1st centenary of Goya...,Brass,"Blay y Fábrega, Miguel",https://content3.cdnprado.net/imagenes/Documen...,1928
6999,Circular carpet,Wool,Real Fábrica de Tapices,https://content3.cdnprado.net/imagenes/Documen...,1928


In [41]:
a=[]
for e in prado_df["year"]:
    if len(e)>4:
        a.append(e)
a

['First third of the XIII',
 'Late XIII',
 'Late XIII',
 'Before  1443',
 'First third of the Último cuarto del siglo XV -...',
 'Late XV',
 'Late XV',
 'Late XV',
 'Early XVI',
 'Early XVI',
 'First quarter of the XVI',
 'First quarter of the XVI',
 'First third of the XVI',
 'First third of the XVI',
 'First third of the XVI',
 'First half of the XVI',
 'First half of the XVI',
 'After  1521',
 'Before  1550',
 'Before  1550',
 'Before  1550',
 'Second third of the XVI',
 'Second third of the XVI',
 'After  1539',
 'Mid-XVIcentury',
 'Mid-XVIcentury',
 'Mid-XVIcentury',
 'Mid-XVIcentury',
 'Second half of the XVI',
 'Second half of the XVI',
 'Second half of the XVI',
 'Second half of the XVI',
 'Second half of the XVI',
 'Second half of the XVI',
 'Second half of the XVI',
 'Second half of the XVI',
 'Second half of the XVI',
 'Second half of the XVI',
 'Second half of the XVI',
 'Primer tercio del siglo...',
 'After  1553',
 'After  1565',
 'Last third of the XVI',
 'After  1575',


In [22]:
prado_df.to_csv("datasets/prado.csv",index=False)

## I will get scrape a website to get the art styles and years

In [18]:
url = "https://artincontext.org/art-periods/"
web= requests.get(url)
soup = BeautifulSoup(web.content, 'html.parser')
    

output= [c.getText().replace(">", "").strip() for c in soup.find_all("td")]

content = output[2:]
diction={}
style=[]
years=[]
i=0
while i< len(content):
    if i==0 or i%2==0:
        style.append(content[i])
    else:
        years.append(content[i])
    i+=1
diction[output[0]]=style
diction[output[1]]=years
    
styles = pd.DataFrame.from_dict(diction)
start_year = []
end_year =[]
for pair in styles["Years"]:
    splited = pair.split(" ")
    start_year.append(int(splited[0]))
    end_year.append(splited[2])
styles["start_year"]=start_year
styles["end_year"]= end_year
styles.drop(columns="Years",inplace=True)
a=[]
for elem in styles["end_year"]:
    if elem=="now":
        elem= elem.replace("now","2022")
        a.append(int(elem))
    else:
        a.append(int(elem))
styles["end_year"]= a
styles

,Art Period,start_year,end_year
0,Romanesque,1000,1150
1,Gothic,1140,1600
2,Renaissance,1495,1527
3,Mannerism,1520,1600
4,Baroque,1600,1725
5,Rococo,1720,1760
6,Neoclassicism,1770,1840
7,Romanticism,1800,1850
8,Realism,1840,1870
9,Pre-Raphaelite,1848,1854


In [24]:
styles.to_csv("datasets/styles.csv",index=False)